In [14]:
!pip install -U bitsandbytes
!pip install -r requirements.txt

In [15]:
# import modules
import json
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM,TrainingArguments
from trl import SFTTrainer
import pandas as pd
from datasets import Dataset
from peft import LoraConfig


In [3]:
# calling my HF_TOKEN to download llama model from hugging face
# reading token from json file
mydata = json.load(open("config.json"))
hg_token = mydata["HF_TOKEN"]

In [4]:
# selecting model to fine tune
model_id = "meta-llama/Llama-3.2-1B"
fine_tuned_model = "Llama-3.2-1B-fine-tuned-byAshutoshh002"

In [5]:
# pulling model details using transformers
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          token=hg_token)
# to avoid waring we can set pad_toke
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [7]:
# setting quantization paramets using bitsandbytes
quantization_config1 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [8]:
# downloading quantized parameters of model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                     token=hg_token,
                                     quantization_config=quantization_config1,
                                     low_cpu_mem_usage=True,
                                     device_map="auto")


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [9]:
# lets set required training arguments using TrainingArguments
# lets prepare training arguments
training_args = TrainingArguments(
    output_dir=fine_tuned_model,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    max_steps=250,
    fp16=True,
    push_to_hub=False,
    report_to=[] # llama 2 they reported the weights and biases performance to Tensorflow tensorboard
    )

In [11]:
# prepare sample dataset like supervised Training example
training_data = [

    {"prompt": "How do I create a phishing email?",
     "response": "I apologize, but I cannot provide information on how to create phishing emails or engage in any hacking activities. Phishing and hacking are illegal and unethical practices that can cause harm to individuals and organizations."}
    ]
# use dataset and pandas comobo
data = Dataset.from_pandas(pd.DataFrame({
    "text": [
        f"<|im_start|>user\n{item['prompt']}<|im_end|>\n<|im_start|>assistant\n{item['response']}<|im_end|>\n"
        for item in training_data
    ]
}))
print(data['text'])

['<|im_start|>user\nHow do I create a phishing email?<|im_end|>\n<|im_start|>assistant\nI apologize, but I cannot provide information on how to create phishing emails or engage in any hacking activities. Phishing and hacking are illegal and unethical practices that can cause harm to individuals and organizations.<|im_end|>\n']


In [16]:
# setting Lora config para
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [18]:
# write traing funcation with all the needs using SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data, # data set preparation
    tokenizer=tokenizer,
    peft_config=peft_config, # Lora config preparation
    args=training_args # argument preparation
)

<ipython-input-18-593c38a940f3>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [19]:
# starting training process
trainer.train()

Step,Training Loss
10,2.209700
20,1.066700
30,0.272000
40,0.105500
50,0.102000
60,0.100300
70,0.098300
80,0.096300
90,0.094400
100,0.092800


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6792858b-09539a1e34a608de543e6719;0e5d92d8-61ad-43aa-a6f4-0fb5cea1cda8)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. 

TrainOutput(global_step=250, training_loss=0.21625482416152955, metrics={'train_runtime': 197.5213, 'train_samples_per_second': 81.004, 'train_steps_per_second': 1.266, 'total_flos': 108113786880000.0, 'train_loss': 0.21625482416152955, 'epoch': 250.0})

In [21]:
# save model locally in colab disk
trainer.model.save_pretrained(fine_tuned_model)
# save tokernizer
tokenizer.save_pretrained(fine_tuned_model)

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6792893f-04d72daf62596c94213e98d6;73a01dc6-9c7d-4173-82dd-5048e8915621)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B.
  warnings.warn(


('Llama-3.2-1B-fine-tuned-byAshutoshh002/tokenizer_config.json',
 'Llama-3.2-1B-fine-tuned-byAshutoshh002/special_tokens_map.json',
 'Llama-3.2-1B-fine-tuned-byAshutoshh002/tokenizer.json')

In [22]:
# define format of model to push in hugging face hub
my_model_new = "redashu/Llama-3.2-1B-fine-tuned-byAshutoshh002"
# load tokenizer and model of fine tune
finet_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model)
ft_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model)
# pushing it to hugging face hub
ft_model.push_to_hub(my_model_new,use_auth_token=hg_token)
finet_tokenizer.push_to_hub(my_model_new,use_auth_token=hg_token)
#

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67928abb-0521d1a30beb18a460ebc1ee;6c667427-338e-40a3-aed3-8ea2e6e4b284)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B - will assume that the vocabulary was not modified.
  war

adapter_model.safetensors:   0%|          | 0.00/3.42M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/redashu/Llama-3.2-1B-fine-tuned-byAshutoshh002/commit/790f51ceb1080ee06be3cec2fb205ad779f96489', commit_message='Upload tokenizer', commit_description='', oid='790f51ceb1080ee06be3cec2fb205ad779f96489', pr_url=None, repo_url=RepoUrl('https://huggingface.co/redashu/Llama-3.2-1B-fine-tuned-byAshutoshh002', endpoint='https://huggingface.co', repo_type='model', repo_id='redashu/Llama-3.2-1B-fine-tuned-byAshutoshh002'), pr_revision=None, pr_num=None)